# ch 15 RNN
- http://pythonkim.tistory.com/57

## 1. Introduction
- <strong>Repeat 모듈</strong>의 추가

### 01. Feature
- <strong>Sequence Data</strong> 를 분석
- by <strong>Time Series</strong> 데이터를 학습/ 훈련/ 테스트
    1. A는 동일하나 $ H_0 , H_1 , H _2..... H_t $ 로 순차적 영향
- 연속 데이터 (Series) 를 대상 : <strong>Repeat 모듈</strong>이 있어서 적합<p></p></br>
<img src="./img/RNN-003.png" align="left">

#### 1) Model
- New state : old state, X 값으로 계산
- RNN 의 구성요소는 아래와 동일 <p></p></br>
<p></br><img src="./img/RNN-007.png" align="left">

#### 2) Function
<p>cf) $k = t-1$ </p></br>
<p>cf) $tanh() : Sigmoid()함수와 유사 $ </p></br>

<p>$ h_t = f_W(h_k, x_t) = tanh(W_h h_k+W_xh x_t) $ </p></br>
<p>$ y_t = W_h h_t $  : 최종적으로는 $ wx $ 형태로 정리</p></br>  


<p></br><img src="./img/RNN-008.png" align="left">

#### 3) Mapping (Language Model)
input | hidden | output(sigmoid) 레이어를 연결</p></br>
<p>Cost/Loss Function 은 Softmax 함수를 활용</p></br>

<img src="./img/RNN-014.png" align="left">

### 02. RNN Application
- 언어모듈 (대화용, 질의응답 Bot)
- 음성인식 Speech Recognition
- Image, Music Caption<p></p>
<img src="./img/Rnn-016.png" align="left">

#### 1) Model 별 용도
- one to one : 단일 입출력
- one to many : Image >> Sequence of words (이미지를 쪼개어서 <strong>여러 단어</strong>로 분석 ex)이미지 캡션)
- many to one : sequence of words >> Sentiment (원하는 <strong>특정 이미지</strong>를 찾는다  ex)Google 이미지 검색)
- many to many 01 : seq of words >> seq of words (Machine Translation)
- many to many 02 : Video classification on frame level</br>
<p>
<img src="./img/RNN-018.png" align = "left">

### 03. Training RNN 모델들
- LSTM (Long Short Term Memory) 가장 대중적인 모델
- GRU (Gated recurrent unit) 우리나라 교수님의 모델

## 2. RNN in TensorFlow
- TensorFlow 0.9 이상에서는 코드가 아래와 같이 변경
- LSTM 공통 01 : Cell 생성
    1. rnn_cell = tf.nn.rnn_cell.<strong>BasicRNNCell</strong>(num_units = rnn_size, input_size = None, )
    1. rnn_cell = tf.nn.rnn_cell.<strong>BasicLSTMCell</strong>(num_units = rnn_size, input_size = None, )
    2. initial_state = rnn_cell.zero_state(batch_size, tf.float32)
    3. initial_state_1 = tf.zeros([batch_size, rnn_cell.state_size])
    4. x_split = tf.split(0, len(char_dic), x_data) 
- LSTM 공통 02 : Cell 구동
    5. outputs, state = tf.nn.rnn(cell = rnn_cell, inputs = x_split, initial_state = initial_state)    

<img src="./img/RNNt-013.png" align="left">

### 01. Cell을 생성
- rnn_size : Output 의 크기값(4)을 입력 <p></p></br>
<img src="./img/RNNt-004.png" align="left">

- input : One-Hot Encoding ->  [[[1,0,0,0]]] shape=(1,1,4)
- Output : Hidden-size = 임의의 값 ->  [[[x,x]]] shape=(1,1,2)
- Input 데이터 형식과 , Hidden-size 만 '매개변수'로 입력하면 작동<p></p>
<img src ="./img/Lab12-RNN01.png" align="left" width=750>

### 02. Cell을 구동
- rnn_size : Output 의 크기값(4)을 입력 <p></p></br>
<img src="./img/RNNt-004.png" align="left">

    outputs, state = tf.nn.rnn(cell = rnn_cell, inputs = x_split, initial_state = initial_state)
- 세부 작동내용 확인 
- time Step size :  모델의 갯수 (여기선 rnn_size와 동일) </br></br><p>
<img src = "./img/RNNt-009.png" align="left">

    x_split = tf.split(0, len(char_dic), x_data) # 가로축으로 4개로 split

### 03. Cost Functions

- logits, targets, weights 별로 사이즈를 맞춰야만 작동이 된다


    logits = tf.reshape(tf.concat(1, outputs), # shape = 1 x 16
                        [-1, rnn_size])        # shape = 4 x 4 (2D [batcha_size * num_decoder_symbols])
    targets = tf.reshape(ground_truth[1:], [-1]) # a shape of [-1] flattens into 1-D
    weights = tf.ones([len(char_dic) * batch_size])

- loss 함수 (개별적 모듈에 적용)
- cost 함수 (batch size 별로 적용)


    loss = tf.nn.seq2seq.sequence_loss_by_example([logits], [targets], [weights])
    cost = tf.reduce_sum(loss) / batch_size
    train_op = tf.train.RMSPropOptimizer(0.01, 0.9).minimize(cost)

- Launch the graph in a session


    with tf.Session() as sess:
        tf.initialize_all_variables().run()
        for i in range(100):
            sess.run(train_op)
            result = sess.run(tf.argmax(logits, 1))
            print(result, [char_rdic[t] for t in result])    